<a href="https://colab.research.google.com/github/FiddiCoder/Movie-Recommender/blob/main/ml_pipeline/Movie_Recommender_DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Import Librerie
import pandas as pd
import numpy as np
import re
from tqdm import tqdm_notebook as tqdm
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import os
import tarfile
from six.moves import urllib
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn import datasets, linear_model,tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from mlxtend.evaluate import bias_variance_decomp
from sklearn.metrics import zero_one_loss
from imblearn.over_sampling import SMOTE

In [ ]:
#@title Download e formattazione dataset film

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/FiddiCoder/Movie-Recommender/main/"
FILE_PATH = os.path.join("dataset")
FILE_URL1 = DOWNLOAD_ROOT + "dataset/movies2"

def fetch_file_data1(file_url1=FILE_URL1, file_path1=FILE_PATH):
    os.makedirs(file_path1, exist_ok=True)
    csv_path1 = os.path.join(file_path1, "movies2")
    urllib.request.urlretrieve(file_url1, csv_path1)

fetch_file_data1()
datapath = os.path.join("dataset","")

with open('./dataset/movies2', encoding = "ISO-8859-1") as content:
    colonne = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'imdb_url']
    generi = ['unknown', 'action', 'adventure', 'animation', 'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy',
          'film-noir',  'horror', 'musical', 'mystery', 'romance', 'sci-fi', 'thriller', 'war', 'western']
    colonneG = colonne + generi
    movies = pd.DataFrame(columns=colonneG)
    i = 0
    for x in content:
        x = x.split("|")
        x[-1] = x[-1][:-1]
        if x[1][-1] == ' ':
            x[1] = x[1][:-1]
        movies.loc[i] = [word if word!='' else "empty" for word in x]
        i = i + 1
movies['movie_id'] = movies['movie_id'].astype('int64')
movies[generi] = movies[generi].astype('category')



In [ ]:
#@title Data cleaning: Tabelle con feature inutili
movies = movies.drop('video_release_date', axis=1)
movies = movies.drop('release_date', axis=1)
movies = movies.drop('imdb_url', axis=1)


In [ ]:
#@title Inserimento colonna target con relativo riempimento
movies['like'] = 0

for i in range(1600):
  if((i % 3) == 0):
    movies.loc[i, 'like'] = 1
  if((i % 8) == 0):
    movies.loc[i , 'like'] = 2
  if((i % 7) == 0):
     movies.loc[i , 'like'] = 3
  if((i % 5) == 0):
    movies.loc[i , 'like'] = 4
  if((i % 6) == 0):
    movies.loc[i , 'like'] = 5


movies.head(100)

In [ ]:
#@title Primo Test: Decision tree
#@markdown Test su target 'like' da 0 a 5 compilato manualmente
#@markdown
#@markdown SI Data cleaning iniziale, drop tabelle
#@markdown
#@markdown NO Feature selection
#@markdown
#@markdown NO Feature Scaling
#@markdown
#@markdown NO Data Balancing
#@markdown
#@markdown NO Undersampling
#@markdown
#@markdown NO Oversampling
X = movies.iloc[:,2:20]
print(X.columns)
y = movies.iloc[:,21:]
print(y.columns)
X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.20, random_state=0)

dtree = tree.DecisionTreeClassifier()
modello = dtree.fit(X_train.values, y_train)

for i in range(1, len(movies)):
    movie = movies.iloc[i, 1]
    if(movie == 'Turbo: A Power Rangers Movie (1997)'):
        movie_1_ind = movies.iloc[i,2:20]
    if(movie == 'Dead Man Walking (1995)'):
        movie_2_ind = movies.iloc[i,2:20]
    if (movie == 'Chasing Amy (1997)'):
        movie_3_ind = movies.iloc[i,2:20]

print("prediction for Turbo: A Power Rangers Movie (1997)")
print(dtree.predict([movie_1_ind]))
print("prediction for Dead Man Walking (1995)")
print(dtree.predict([movie_2_ind]))
print("prediction for Chasing Amy (1997)")
print(dtree.predict([movie_3_ind]))
print("\n")

score = cross_val_score(modello, X_train, y_train, cv = 10)
print("Cross validation: \n",score)
print("Testing score: {}".format(modello.score(X_test, y_test)))
print("\n")

print('-------------------------------')
print('\n')
print('\n')
print('\n')

dtree = tree.DecisionTreeClassifier(random_state=123)
dtree.fit(X_train.values,y_train.values)
predict = dtree.predict(X_test.values)

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        dtree, X_train.values, y_train.values, X_test.values, y_test.values,
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias) #underfitting altissimo
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,predict))


### After Pruning ###
print('/n')
dtree_prnd = tree.DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=123)
dtree_prnd.fit(X_train.values,y_train.values)
predict=dtree_prnd.predict(X_test.values)

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        dtree_prnd, X_train.values, y_train.values, X_test.values, y_test.values,
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss--After pruning: %.3f' % avg_expected_loss)
print('Average bias--After pruning: %.3f' % avg_bias)
print('Average variance--After pruning: %.3f' % avg_var)
print('Sklearn 0-1 loss--After pruning: %.3f' % zero_one_loss(y_test,predict))

In [ ]:
#@title Download txt utenti e formattazione in dataset
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/FiddiCoder/Movie-Recommender/main/"
FILE_PATH = os.path.join("dataset")
FILE_URL1 = DOWNLOAD_ROOT + "dataset/users.user"

def fetch_file_data1(file_url1=FILE_URL1, file_path1=FILE_PATH):
    os.makedirs(file_path1, exist_ok=True)
    csv_path1 = os.path.join(file_path1, "users.user")
    urllib.request.urlretrieve(file_url1, csv_path1)

fetch_file_data1()
datapath = os.path.join("dataset","")

colonne_user = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
users = pd.read_csv('./dataset/users.user', sep='|', names=colonne_user)


In [ ]:
#@title Data cleaning, trasformazione variabili da string a boolean

#Drop feature inutile
users = users.drop('zip_code', axis=1)

#Data cleaning = Eliminazione righe vuote
users = users[users['gender'] != 'empty']
users = users[users['occupation'] != 'empty']

#Colonne occupation e gender trasformate in più colonne coi rispettivi nomi delle variabili per dare loro valori booleani
users['occupation'] = users['occupation'].astype('category')
users['gender'] = users['gender'].astype('category')
one_hot_occ = users.occupation.str.get_dummies()
users = users.drop('occupation',axis=1)
users = users.join(one_hot_occ,how='inner')
one_hot_g = users.gender.str.get_dummies()
users = users.drop('gender',axis=1)
users = users.join(one_hot_g,how='inner')
users.head(5)

,user_id,age,administrator,artist,doctor,educator,engineer,entertainment,executive,healthcare,...,other,programmer,retired,salesman,scientist,student,technician,writer,F,M
0,1,24,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
1,2,53,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,3,23,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,4,24,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,5,33,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [ ]:
#@title Download dataset chiave per joinare utenti e film
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/FiddiCoder/Movie-Recommender/main/"
FILE_PATH = os.path.join("dataset")
FILE_URL1 = DOWNLOAD_ROOT + "dataset/usersmovies.data"

def fetch_file_data1(file_url1=FILE_URL1, file_path1=FILE_PATH):
    os.makedirs(file_path1, exist_ok=True)
    csv_path1 = os.path.join(file_path1, "usersmovies.data")
    urllib.request.urlretrieve(file_url1, csv_path1)

fetch_file_data1()
datapath = os.path.join("dataset","")

colonnekey = ['user_id', 'movie_id', 'rating', 'timestamp']
key = pd.read_csv('./dataset/usersmovies.data', sep='\t', names=colonnekey)

#Drop feature inutile
key = key.drop('timestamp', axis=1)
key.head(50)

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
5,298,474,4
6,115,265,2
7,253,465,5
8,305,451,3
9,6,86,3


In [ ]:
#@title Drop vecchia colonna target 'like', da ora utilizzeremo 'rating'
movies = movies.drop('like', axis=1)

In [ ]:
#@title Join dataset utenti e film
final_dataset = key.merge(movies, left_on='movie_id', right_on='movie_id', how='inner')
final_dataset = final_dataset.merge(users, left_on='user_id', right_on='user_id', how='inner')

#Spostiamo la colonna target in ultima posizione
sposta_colonna = final_dataset.pop("rating")
final_dataset.insert(46, "rating", sposta_colonna)

final_dataset.head(10)

,user_id,movie_id,movie_title,unknown,action,adventure,animation,children,comedy,crime,...,programmer,retired,salesman,scientist,student,technician,writer,F,M,rating
0,196,242,Kolya (1996),0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,3
1,196,257,Men in Black (1997),0,1,1,0,0,1,0,...,0,0,0,0,0,0,1,0,1,2
2,196,111,"Truth About Cats & Dogs, The (1996)",0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,4
3,196,25,"Birdcage, The (1996)",0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,4
4,196,382,"Adventures of Priscilla, Queen of the Desert, ...",0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,4
5,196,202,Groundhog Day (1993),0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,3
6,196,153,"Fish Called Wanda, A (1988)",0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,5
7,196,286,"English Patient, The (1996)",0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,5
8,196,66,While You Were Sleeping (1995),0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,3
9,196,845,That Thing You Do! (1996),0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,4


In [ ]:
#@title Secondo Test: Decision tree
#@markdown Nuovo dataset: Ottenuto da join tra film e utenti
#@markdown
#@markdown Test su target 'rating' da 0 a 5, valori nel dataset ''ufficiale''
#@markdown
#@markdown SI Data cleaning iniziale, drop tabelle
#@markdown
#@markdown NO Feature selection
#@markdown
#@markdown NO Feature Scaling
#@markdown
#@markdown NO Data Balancing
#@markdown
#@markdown NO Undersampling
#@markdown
#@markdown NO Oversampling

#@markdown SEZIONI DI CODICE SUCCESSIVE: Abbiamo di conseguenza pensato di trasformare la variabile target in un booleano, affinchè potesse risultare più semplice la classificazione tramite decision tree
X = final_dataset.iloc[:,3:46]
print(X.columns)
y = final_dataset.iloc[:,46:]
print(y.columns)
X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.20, random_state=0)

dtree = tree.DecisionTreeClassifier()
modello = dtree.fit(X_train.values, y_train)

for i in range(1, len(final_dataset)):
    movie = final_dataset.iloc[i, 2]
    if(movie == 'Raising Arizona (1987)'):
        movie_1_ind = final_dataset.iloc[i, 3:46]
    if(movie == 'Little Big League (1994)'):
        movie_2_ind = final_dataset.iloc[i, 3:46]
    if (movie == 'Mighty Aphrodite (1995)'):
        movie_3_ind = final_dataset.iloc[i, 3:46]
print("prediction for Raising Arizona (1987))")
predict = dtree.predict([movie_1_ind])
print(predict)
print("prediction for Dead Man Walking (1995)")
predict = dtree.predict([movie_2_ind])
print(predict)
print("prediction for Chasing Amy (1997)")
predict = dtree.predict([movie_3_ind])
print(predict)
print("\n")


score = cross_val_score(modello, X_train, y_train, cv = 10)
print("cross validation scores: \n",score)
print("testing score: {}".format(modello.score(X_test, y_test)))
print("\n")


predict = dtree.predict(X_test)
print('Accuracy modello: {0:0.4f}'.
format(accuracy_score(y_test, predict)))

cm = confusion_matrix(y_test, predict)
print('Confusion matrix\n\n', cm)

print('-------------------------------')
print('\n')
print('\n')
print('\n')

dtree = tree.DecisionTreeClassifier(random_state=123)
dtree.fit(X_train.values,y_train.values)
predict = dtree.predict(X_test.values)

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        dtree, X_train.values, y_train.values, X_test.values, y_test.values,
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias) #underfitting altissimo
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,predict))


### After Pruning ###
print('/n')
dtree_prnd = tree.DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=123)
dtree_prnd.fit(X_train.values,y_train.values)
predict=dtree_prnd.predict(X_test.values)

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        dtree_prnd, X_train.values, y_train.values, X_test.values, y_test.values,
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss--After pruning: %.3f' % avg_expected_loss)
print('Average bias--After pruning: %.3f' % avg_bias)
print('Average variance--After pruning: %.3f' % avg_var)
print('Sklearn 0-1 loss--After pruning: %.3f' % zero_one_loss(y_test,predict))

Index(['unknown', 'action', 'adventure', 'animation', 'children', 'comedy',
       'crime', 'documentary', 'drama', 'fantasy', 'film-noir', 'horror',
       'musical', 'mystery', 'romance', 'sci-fi', 'thriller', 'war', 'western',
       'age', 'administrator', 'artist', 'doctor', 'educator', 'engineer',
       'entertainment', 'executive', 'healthcare', 'homemaker', 'lawyer',
       'librarian', 'marketing', 'none', 'other', 'programmer', 'retired',
       'salesman', 'scientist', 'student', 'technician', 'writer', 'F', 'M'],
      dtype='object')
Index(['rating'], dtype='object')
prediction for Raising Arizona (1987))
[4]
prediction for Dead Man Walking (1995)
[3]
prediction for Chasing Amy (1997)
[3]


cross validation scores: 
 [0.331875 0.33575  0.322875 0.338625 0.333125 0.343625 0.343625 0.332375
 0.328625 0.327375]
testing score: 0.33725


Accuracy modello: 0.3372
Confusion matrix

 [[ 321  132  348  312  104]
 [ 172  302  779  728  206]
 [ 294  597 1979 2006  611]
 [ 257  552 1

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


Average expected loss: 0.739
Average bias: 14664.338
Average variance: 0.335
Sklearn 0-1 loss: 0.662
/n
Average expected loss--After pruning: 0.662
Average bias--After pruning: 13243.892
Average variance--After pruning: 0.007
Sklearn 0-1 loss--After pruning: 0.652


In [ ]:
#@title Trasformazione colonna target in booleana

#Vecchia implementazione, modificata dopo Quarto Test

final_dataset['rating'] = final_dataset['rating'].replace(0,0)
final_dataset['rating'] = final_dataset['rating'].replace(1,0)
final_dataset['rating'] = final_dataset['rating'].replace(2,0)
final_dataset['rating'] = final_dataset['rating'].replace(3,1)
final_dataset['rating'] = final_dataset['rating'].replace(4,1)
final_dataset['rating'] = final_dataset['rating'].replace(5,1)
final_dataset.head(10)


,user_id,movie_id,movie_title,unknown,action,adventure,animation,children,comedy,crime,...,programmer,retired,salesman,scientist,student,technician,writer,F,M,rating
0,196,242,Kolya (1996),0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,1
1,196,257,Men in Black (1997),0,1,1,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
2,196,111,"Truth About Cats & Dogs, The (1996)",0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,1
3,196,25,"Birdcage, The (1996)",0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,1
4,196,382,"Adventures of Priscilla, Queen of the Desert, ...",0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,1
5,196,202,Groundhog Day (1993),0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,1
6,196,153,"Fish Called Wanda, A (1988)",0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,1
7,196,286,"English Patient, The (1996)",0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1
8,196,66,While You Were Sleeping (1995),0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,1
9,196,845,That Thing You Do! (1996),0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,1


In [ ]:
#@title Terzo Test: Decision tree
#@markdown Test su target 'rating' da 0 a 1 --> (0,1,2 = 0),(3,4,5=1)
#@markdown
#@markdown SI Data cleaning iniziale, drop tabelle
#@markdown
#@markdown NO Feature selection
#@markdown
#@markdown SI Feature Scaling, su variaile target
#@markdown
#@markdown NO Data Balancing
#@markdown
#@markdown NO Undersampling
#@markdown
#@markdown NO Oversampling
#@markdown
#@markdown SEZIONI DI CODICE SUCCESSIVE: Abbiamo pensato di fare Feature Selection per migliorare ulteriormente lo score.
X = final_dataset.iloc[:,3:46]
print(X.columns)
y = final_dataset.iloc[:,46:]
print(y.columns)
X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.20, random_state=0)

dtree = tree.DecisionTreeClassifier()
modello = dtree.fit(X_train.values, y_train)

for i in range(1, len(final_dataset)):
    movie = final_dataset.iloc[i, 2]
    if(movie == 'Raising Arizona (1987)'):
        movie_1_ind = final_dataset.iloc[i, 3:46]
    if(movie == 'Little Big League (1994)'):
        movie_2_ind = final_dataset.iloc[i, 3:46]
    if (movie == 'Mighty Aphrodite (1995)'):
        movie_3_ind = final_dataset.iloc[i, 3:46]
print("prediction for Raising Arizona (1987))")
predict = dtree.predict([movie_1_ind])
print(predict)
print("prediction for Dead Man Walking (1995)")
predict = dtree.predict([movie_2_ind])
print(predict)
print("prediction for Chasing Amy (1997)")
predict = dtree.predict([movie_3_ind])
print(predict)
print("\n")


score = cross_val_score(modello, X_train, y_train, cv = 10)
print("Cross validation: \n",score)
print("Testing score: {}".format(modello.score(X_test, y_test)))
print("\n")


predict = dtree.predict(X_test)
print('Accuracy modello: {0:0.4f}'.
format(accuracy_score(y_test, predict)))

cm = confusion_matrix(y_test, predict)
print('Confusion matrix\n\n', cm)

print('-------------------------------')
print('\n')
print('\n')
print('\n')

dtree = tree.DecisionTreeClassifier(random_state=123)
dtree.fit(X_train.values,y_train.values)
predict = dtree.predict(X_test.values)

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        dtree, X_train.values, y_train.values, X_test.values, y_test.values,
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias) #underfitting altissimo
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,predict))


### After Pruning ###
print('/n')
dtree_prnd = tree.DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=123)
dtree_prnd.fit(X_train.values,y_train.values)
predict=dtree_prnd.predict(X_test.values)

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        dtree_prnd, X_train.values, y_train.values, X_test.values, y_test.values,
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss--After pruning: %.3f' % avg_expected_loss)
print('Average bias--After pruning: %.3f' % avg_bias)
print('Average variance--After pruning: %.3f' % avg_var)
print('Sklearn 0-1 loss--After pruning: %.3f' % zero_one_loss(y_test,predict))


Index(['unknown', 'action', 'adventure', 'animation', 'children', 'comedy',
       'crime', 'documentary', 'drama', 'fantasy', 'film-noir', 'horror',
       'musical', 'mystery', 'romance', 'sci-fi', 'thriller', 'war', 'western',
       'age', 'administrator', 'artist', 'doctor', 'educator', 'engineer',
       'entertainment', 'executive', 'healthcare', 'homemaker', 'lawyer',
       'librarian', 'marketing', 'none', 'other', 'programmer', 'retired',
       'salesman', 'scientist', 'student', 'technician', 'writer', 'F', 'M'],
      dtype='object')
Index(['rating'], dtype='object')
prediction for Raising Arizona (1987))
[1]
prediction for Dead Man Walking (1995)
[1]
prediction for Chasing Amy (1997)
[1]


Cross validation: 
 [0.78175  0.780375 0.7855   0.7835   0.786625 0.780625 0.783375 0.786875
 0.7835   0.781625]
Testing score: 0.78625


Accuracy modello: 0.7863
Confusion matrix

 [[  797  2607]
 [ 1668 14928]]
-------------------------------








/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


Average expected loss: 0.258
Average bias: 4678.347
Average variance: 0.091
Sklearn 0-1 loss: 0.214
/n
Average expected loss--After pruning: 0.175
Average bias--After pruning: 3507.557
Average variance--After pruning: 0.000
Sklearn 0-1 loss--After pruning: 0.167


In [ ]:
#@title Feature Selection

#Trasformazione variabili categoriche affinchè compaiano nella matrice di correlazione
le = LabelEncoder()
final_dataset['unknown'] = le.fit_transform(final_dataset['unknown'])
final_dataset['action'] = le.fit_transform(final_dataset['action'])
final_dataset['adventure'] = le.fit_transform(final_dataset['adventure'])
final_dataset['animation'] = le.fit_transform(final_dataset['animation'])
final_dataset['children'] = le.fit_transform(final_dataset['children'])
final_dataset['comedy'] = le.fit_transform(final_dataset['comedy'])
final_dataset['crime'] = le.fit_transform(final_dataset['crime'])
final_dataset['documentary'] = le.fit_transform(final_dataset['documentary'])
final_dataset['drama'] = le.fit_transform(final_dataset['drama'])
final_dataset['fantasy'] = le.fit_transform(final_dataset['fantasy'])
final_dataset['film-noir'] = le.fit_transform(final_dataset['film-noir'])
final_dataset['horror'] = le.fit_transform(final_dataset['horror'])
final_dataset['musical'] = le.fit_transform(final_dataset['musical'])
final_dataset['mystery'] = le.fit_transform(final_dataset['mystery'])
final_dataset['romance'] = le.fit_transform(final_dataset['romance'])
final_dataset['sci-fi'] = le.fit_transform(final_dataset['sci-fi'])
final_dataset['thriller'] = le.fit_transform(final_dataset['thriller'])
final_dataset['war'] = le.fit_transform(final_dataset['war'])
final_dataset['western'] = le.fit_transform(final_dataset['western'])

#Matrice correlazione feature
sns.set(rc = {'figure.figsize':(48,24)})
sns.heatmap(final_dataset.corr(), annot = True, fmt='.2g',cmap= 'coolwarm')


In [ ]:
#@title Ordinamento correlazioni con feature target
# Calcola la correlazione tra le feature e la colonna del rating
correlation_matrix = final_dataset.corr()
rating_correlation = correlation_matrix['rating'].drop('rating').drop('movie_id').drop('user_id')  # Rimuovi la correlazione con se stessa

# Ordina le feature in base alla correlazione con il rating
sorted_features = rating_correlation.abs().sort_values(ascending=False)

# Stampa le feature ordinate per correlazione con il rating
print(sorted_features)


<ipython-input-60-a8504e615868>:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = final_dataset.corr()


drama            0.083915
healthcare       0.080840
age              0.057271
comedy           0.052966
war              0.046359
horror           0.045483
executive        0.039378
writer           0.037685
educator         0.034889
film-noir        0.032470
romance          0.029448
children         0.028512
administrator    0.022354
fantasy          0.021812
action           0.019794
scientist        0.017199
mystery          0.016042
crime            0.015191
western          0.014460
engineer         0.014318
librarian        0.013515
none             0.013513
entertainment    0.012831
doctor           0.012356
lawyer           0.010996
M                0.010759
F                0.010759
animation        0.010050
homemaker        0.009999
student          0.009345
technician       0.008424
adventure        0.008331
other            0.006901
retired          0.006122
programmer       0.005067
documentary      0.004705
artist           0.003584
thriller         0.002279
salesman    

In [ ]:
#@title Prendiamo le migliori 24 feature

final_dataset = final_dataset.drop('marketing',axis=1)
final_dataset = final_dataset.drop('unknown',axis=1)
final_dataset = final_dataset.drop('musical',axis=1)
final_dataset = final_dataset.drop('sci-fi',axis=1)
final_dataset = final_dataset.drop('salesman',axis=1)
final_dataset = final_dataset.drop('artist',axis=1)
final_dataset = final_dataset.drop('retired',axis=1)
final_dataset = final_dataset.drop('other',axis=1)
final_dataset = final_dataset.drop('adventure',axis=1)
final_dataset = final_dataset.drop('technician',axis=1)
final_dataset = final_dataset.drop('student',axis=1)
final_dataset = final_dataset.drop('homemaker',axis=1)
final_dataset = final_dataset.drop('animation',axis=1)
final_dataset = final_dataset.drop('F',axis=1)
final_dataset = final_dataset.drop('M',axis=1)
final_dataset = final_dataset.drop('lawyer',axis=1)
final_dataset = final_dataset.drop('programmer',axis=1)
final_dataset = final_dataset.drop('documentary',axis=1)
final_dataset = final_dataset.drop('thriller',axis=1)

#Scartiamo le righe che hanno tutti i generi vuoti
final_dataset = final_dataset.loc[~((final_dataset['drama'] == 0) & (final_dataset['comedy'] == 0) & (final_dataset['war'] == 0)
 & (final_dataset['horror'] == 0)  & (final_dataset['film-noir'] == 0) & (final_dataset['romance'] == 0)
  & (final_dataset['children'] == 0) & (final_dataset['fantasy'] == 0) & (final_dataset['action'] == 0)
  &  (final_dataset['mystery'] == 0) & (final_dataset['crime'] == 0) & (final_dataset['western'] == 0))
 ]

#Scartiamo le righe che hanno tutti i dati dell'utente vuoti
final_dataset = final_dataset.loc[~((final_dataset['healthcare'] == 0) & (final_dataset['age'] == 0) & (final_dataset['executive'] == 0)
 & (final_dataset['writer'] == 0)  & (final_dataset['educator'] == 0) & (final_dataset['administrator'] == 0)
  & (final_dataset['scientist'] == 0) & (final_dataset['engineer'] == 0) & (final_dataset['librarian'] == 0)
  & (final_dataset['doctor'] == 0) & (final_dataset['entertainment'] == 0) & (final_dataset['none'] == 0))]


final_dataset.head(100)


,user_id,movie_id,movie_title,action,children,comedy,crime,drama,fantasy,film-noir,...,educator,engineer,entertainment,executive,healthcare,librarian,none,scientist,writer,rating
0,196,242,Kolya (1996),0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,196,257,Men in Black (1997),1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,196,111,"Truth About Cats & Dogs, The (1996)",0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,196,25,"Birdcage, The (1996)",0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,196,382,"Adventures of Priscilla, Queen of the Desert, ...",0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,63,284,Tin Cup (1996),0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
99,63,13,Mighty Aphrodite (1995),0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
100,63,225,101 Dalmatians (1996),0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
101,63,948,Booty Call (1997),0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
#@title Quarto test: Decision tree

#@markdown SI Data cleaning iniziale, drop tabelle
#@markdown
#@markdown SI Feature selection, prese le 24 variabili indipendenti più correlate con la variabile target
#@markdown
#@markdown SI Feature Scaling
#@markdown
#@markdown NO Data Balancing
#@markdown
#@markdown NO Undersampling
#@markdown
#@markdown NO Oversampling
#@markdown
#@markdown SEZIONI DI CODICE SUCCESSIVE:Si nota un'occorrenza sbilanciata di variabile target=0 rispetto a target=1, Si è pensato dunque di bilanciare i dati attraverso undersampling/oversampling

X = final_dataset.iloc[:,3:27]
print(X.columns)
y = final_dataset.iloc[:,27:]
print(y.columns)
X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.20, random_state=0)

dtree = tree.DecisionTreeClassifier()
modello = dtree.fit(X_train.values, y_train)

for i in range(1, len(final_dataset)):
    movie = final_dataset.iloc[i, 2]
    if(movie == 'Raising Arizona (1987)'):
        movie_1_ind = final_dataset.iloc[i, 3:27]
    if(movie == 'Little Big League (1994)'):
        movie_2_ind = final_dataset.iloc[i, 3:27]
    if (movie == 'Mighty Aphrodite (1995)'):
        movie_3_ind = final_dataset.iloc[i, 3:27]
print("prediction for Raising Arizona (1987))")
predict = dtree.predict([movie_1_ind])
print(predict)
print("prediction for Dead Man Walking (1995)")
predict = dtree.predict([movie_2_ind])
print(predict)
print("prediction for Chasing Amy (1997)")
predict = dtree.predict([movie_3_ind])
print(predict)
print("\n")


score = cross_val_score(modello, X_train, y_train, cv = 10)
print("Cross validation: \n",score)
print("Testing score: {}".format(modello.score(X_test, y_test)))
print("\n")


predict = dtree.predict(X_test)
print('Accuracy modello: {0:0.4f}'.
format(accuracy_score(y_test, predict)))

cm = confusion_matrix(y_test, predict)
print('Confusion matrix\n\n', cm)

print('-------------------------------')
print('\n')
print('\n')
print('\n')

dtree = tree.DecisionTreeClassifier(random_state=123)
dtree.fit(X_train.values,y_train.values)
predict = dtree.predict(X_test.values)

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        dtree, X_train.values, y_train.values, X_test.values, y_test.values,
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias) #underfitting altissimo
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,predict))


### After Pruning ###
print('/n')
dtree_prnd = tree.DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=123)
dtree_prnd.fit(X_train.values,y_train.values)
predict=dtree_prnd.predict(X_test.values)

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        dtree_prnd, X_train.values, y_train.values, X_test.values, y_test.values,
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss--After pruning: %.3f' % avg_expected_loss)
print('Average bias--After pruning: %.3f' % avg_bias)
print('Average variance--After pruning: %.3f' % avg_var)
print('Sklearn 0-1 loss--After pruning: %.3f' % zero_one_loss(y_test,predict))

Index(['action', 'children', 'comedy', 'crime', 'drama', 'fantasy',
       'film-noir', 'horror', 'mystery', 'romance', 'war', 'western', 'age',
       'administrator', 'doctor', 'educator', 'engineer', 'entertainment',
       'executive', 'healthcare', 'librarian', 'none', 'scientist', 'writer'],
      dtype='object')
Index(['rating'], dtype='object')
prediction for Raising Arizona (1987))
[1]
prediction for Dead Man Walking (1995)
[1]
prediction for Chasing Amy (1997)
[1]


Cross validation: 
 [0.80730249 0.81094075 0.81159044 0.81094075 0.8100065  0.80896686
 0.8100065  0.80987654 0.80909682 0.81221572]
Testing score: 0.8066947346535682


Accuracy modello: 0.8067
Confusion matrix

 [[  480  2917]
 [  802 15040]]
-------------------------------








/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


Average expected loss: 0.225
Average bias: 4123.427
Average variance: 0.047
Sklearn 0-1 loss: 0.193
/n
Average expected loss--After pruning: 0.182
Average bias--After pruning: 3492.089
Average variance--After pruning: 0.000
Sklearn 0-1 loss--After pruning: 0.172


In [ ]:
#@title Quinto test: Naive-Bayes
#@markdown SI Data cleaning iniziale, drop tabelle
#@markdown
#@markdown SI Feature selection, prese le 24 variabili indipendenti più correlate con la variabile target
#@markdown
#@markdown SI Feature Scaling
#@markdown
#@markdown NO Data Balancing
#@markdown
#@markdown NO Undersampling
#@markdown
#@markdown NO Oversampling
#@markdown
#@markdown
#@markdown SEZIONI DI CODICE SUCCESSIVE:Si nota un'occorrenza sbilanciata di variabile target=0 rispetto a target=1, Si è pensato dunque di bilanciare i dati attraverso undersampling/oversampling

X = final_dataset.iloc[:,3:27]
print(X.columns)
y = final_dataset.iloc[:,27:]
print(y.columns)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=0
)

model = GaussianNB()

modello = model.fit(X_train, y_train)

for i in range(1, len(final_dataset)):
    movie = final_dataset.iloc[i, 2]
    if(movie == 'Raising Arizona (1987)'):
        movie_1_ind = final_dataset.iloc[i, 3:27]
        movie_1_real = final_dataset.iloc[i, 27]
    if(movie == 'Little Big League (1994)'):
        movie_2_ind = final_dataset.iloc[i, 3:27]
        movie_2_real = final_dataset.iloc[i, 27]
    if (movie == 'Mighty Aphrodite (1995)'):
        movie_3_ind = final_dataset.iloc[i, 3:27]
        movie_3_real = final_dataset.iloc[i, 27]
print("prediction for Raising Arizona (1987))")
predict = model.predict([movie_1_ind])
print(predict)
print("Real value")
print(movie_1_real)
print("prediction for Dead Man Walking (1995)")
predict = model.predict([movie_2_ind])
print(predict)
print("Real value")
print(movie_2_real)
print("prediction for Chasing Amy (1997)")
predict = model.predict([movie_3_ind])
print(predict)
print("Real value")
print(movie_3_real)
print("\n")

score = cross_val_score(modello, X_train, y_train, cv = 10)
print("Cross validation: \n",score)
print("Testing score: {}".format(modello.score(X_test, y_test)))
print("\n")


y_pred = model.predict(X_test)
accuray = accuracy_score(y_pred, y_test)
print("Accuracy:", accuray)

cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix\n\n', cm)



Index(['action', 'children', 'comedy', 'crime', 'drama', 'fantasy',
       'film-noir', 'horror', 'mystery', 'romance', 'war', 'western', 'age',
       'administrator', 'doctor', 'educator', 'engineer', 'entertainment',
       'executive', 'healthcare', 'librarian', 'none', 'scientist', 'writer'],
      dtype='object')
Index(['rating'], dtype='object')


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


prediction for Raising Arizona (1987))
[1]
Real value
1
prediction for Dead Man Walking (1995)
[0]
Real value
1
prediction for Chasing Amy (1997)
[1]
Real value
1




/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usi

Cross validation: 
 [0.71387734 0.72570166 0.71738565 0.72284304 0.70929175 0.71163093
 0.71578947 0.71994802 0.71786875 0.72501624]
Testing score: 0.7121991787514944


Accuracy: 0.7121991787514944
Confusion matrix

 [[ 1057  2340]
 [ 3197 12645]]


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#@title Conta numero occorrenze valori variabile target
# Conta il numero di righe in cui il valore della colonna è 1
count_1 = len(final_dataset[final_dataset['rating'] == 1])

# Conta il numero di righe in cui il valore della colonna è 0
count_0 = len(final_dataset[final_dataset['rating'] == 0])

# Stampa i risultati
print("Numero di righe con valore 1:", count_1)
print("Numero di righe con valore 0:", count_0)

Numero di righe con valore 1: 79484
Numero di righe con valore 0: 16709


In [ ]:
#@title Test aggiuntivo: Oversampling

#@markdown SI Data cleaning iniziale, drop tabelle
#@markdown
#@markdown SI Feature selection, prese le 24 variabili indipendenti più correlate con la variabile target
#@markdown
#@markdown SI Feature Scaling
#@markdown
#@markdown SI Data Balancing, Oversampling
#@markdown
#@markdown NO Undersampling
#@markdown
#@markdown SI Oversampling


X = final_dataset.iloc[:,3:27]
print(X.columns)
y = final_dataset.iloc[:,27:]
print(y.columns)
x_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.20, random_state=0)

smote = SMOTE()

x_train_oversampled, y_train_oversampled = smote.fit_resample(x_train, y_train)

dtree = tree.DecisionTreeClassifier()
modello = dtree.fit(x_train_oversampled.values, y_train_oversampled)

for i in range(1, len(final_dataset)):
    movie = final_dataset.iloc[i, 2]
    if(movie == 'Raising Arizona (1987)'):
        movie_1_ind = final_dataset.iloc[i, 3:27]
    if(movie == 'Little Big League (1994)'):
        movie_2_ind = final_dataset.iloc[i, 3:27]
    if (movie == 'Mighty Aphrodite (1995)'):
        movie_3_ind = final_dataset.iloc[i, 3:27]
print("prediction for Raising Arizona (1987))")
predict = dtree.predict([movie_1_ind])
print(predict)
print("prediction for Dead Man Walking (1995)")
predict = dtree.predict([movie_2_ind])
print(predict)
print("prediction for Chasing Amy (1997)")
predict = dtree.predict([movie_3_ind])
print(predict)
print("\n")


score = cross_val_score(modello, x_train_oversampled, y_train_oversampled, cv = 10)
print("Cross validation: \n",score)
print("Testing score: {}".format(modello.score(X_test, y_test)))
print("\n")


predict = dtree.predict(X_test)
print('Accuracy modello: {0:0.4f}'.
format(accuracy_score(y_test, predict)))

cm = confusion_matrix(y_test, predict)
print('Confusion matrix\n\n', cm)

print('-------------------------------')
print('\n')
print('\n')
print('\n')

dtree = tree.DecisionTreeClassifier(random_state=123)
dtree.fit(x_train_oversampled.values,y_train_oversampled.values)
predict = dtree.predict(X_test.values)

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        dtree, x_train_oversampled.values, y_train_oversampled.values, X_test.values, y_test.values,
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias) #underfitting altissimo
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,predict))


### After Pruning ###
dtree_prnd = tree.DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=123)
dtree_prnd.fit(x_train_oversampled.values,y_train_oversampled.values)
predict=dtree_prnd.predict(X_test.values)

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        dtree_prnd, x_train_oversampled.values, y_train_oversampled.values, X_test.values, y_test.values,
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss--After pruning: %.3f' % avg_expected_loss)
print('Average bias--After pruning: %.3f' % avg_bias)
print('Average variance--After pruning: %.3f' % avg_var)
print('Sklearn 0-1 loss--After pruning: %.3f' % zero_one_loss(y_test,predict))

In [ ]:
#@title Undersampling
# Determina il numero di righe da eliminare casualmente
num_rows_to_drop = 60000  # Specifica il numero di righe da eliminare

# Seleziona solo le righe con rating = 1 e determina l'indice di queste righe
rows_to_drop = final_dataset.loc[final_dataset['rating'] == 1].index

# Genera un array casuale di indici di riga da eliminare
random_rows = np.random.choice(rows_to_drop, size=num_rows_to_drop, replace=False)

# Elimina le righe casuali dal DataFrame
final_dataset.drop(random_rows, inplace=True)


# Conta il numero di righe in cui il valore della colonna è 1
count_1 = len(final_dataset[final_dataset['rating'] == 1])

# Conta il numero di righe in cui il valore della colonna è 0
count_0 = len(final_dataset[final_dataset['rating'] == 0])

# Stampa i risultati
print("Numero di righe con valore 1:", count_1)
print("Numero di righe con valore 0:", count_0)



Numero di righe con valore 1: 19484
Numero di righe con valore 0: 16709


In [ ]:
#@title Sesto test: Decision Tree
#@markdown SI Data cleaning iniziale, drop tabelle
#@markdown
#@markdown SI Feature selection, prese le 24 variabili indipendenti più correlate con la variabile target
#@markdown
#@markdown SI Feature Scaling
#@markdown
#@markdown SI Data Balancing, Undersampling
#@markdown
#@markdown SI Undersampling
#@markdown
#@markdown NO Oversampling
X = final_dataset.iloc[:,3:27]
print(X.columns)
y = final_dataset.iloc[:,27:]
print(y.columns)
X_train, X_test, y_train, y_test = train_test_split(X, y,
test_size=0.20, random_state=0)

dtree = tree.DecisionTreeClassifier()
modello = dtree.fit(X_train.values, y_train)

for i in range(1, len(final_dataset)):
    movie = final_dataset.iloc[i, 2]
    if(movie == 'Raising Arizona (1987)'):
        movie_1_ind = final_dataset.iloc[i, 3:27]
    if(movie == 'Little Big League (1994)'):
        movie_2_ind = final_dataset.iloc[i, 3:27]
    if (movie == 'Mighty Aphrodite (1995)'):
        movie_3_ind = final_dataset.iloc[i, 3:27]
print("prediction for Raising Arizona (1987))")
predict = dtree.predict([movie_1_ind])
print(predict)
print("prediction for Dead Man Walking (1995)")
predict = dtree.predict([movie_2_ind])
print(predict)
print("prediction for Chasing Amy (1997)")
predict = dtree.predict([movie_3_ind])
print(predict)
print("\n")


score = cross_val_score(modello, X_train, y_train, cv = 10)
print("Cross validation: \n",score)
print("Testing score: {}".format(modello.score(X_test, y_test)))
print("\n")


predict = dtree.predict(X_test)
print('Accuracy modello: {0:0.4f}'.
format(accuracy_score(y_test, predict)))

cm = confusion_matrix(y_test, predict)
print('Confusion matrix\n\n', cm)

print('-------------------------------')
print('\n')
print('\n')
print('\n')

dtree = tree.DecisionTreeClassifier(random_state=123)
dtree.fit(X_train.values,y_train.values)
predict = dtree.predict(X_test.values)

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        dtree, X_train.values, y_train.values, X_test.values, y_test.values,
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias) #underfitting altissimo
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,predict))


### After Pruning ###
dtree_prnd = tree.DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=123)
dtree_prnd.fit(X_train.values,y_train.values)
predict=dtree_prnd.predict(X_test.values)

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        dtree_prnd, X_train.values, y_train.values, X_test.values, y_test.values,
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss--After pruning: %.3f' % avg_expected_loss)
print('Average bias--After pruning: %.3f' % avg_bias)
print('Average variance--After pruning: %.3f' % avg_var)
print('Sklearn 0-1 loss--After pruning: %.3f' % zero_one_loss(y_test,predict))

Index(['action', 'children', 'comedy', 'crime', 'drama', 'fantasy',
       'film-noir', 'horror', 'mystery', 'romance', 'war', 'western', 'age',
       'administrator', 'doctor', 'educator', 'engineer', 'entertainment',
       'executive', 'healthcare', 'librarian', 'none', 'scientist', 'writer'],
      dtype='object')
Index(['rating'], dtype='object')
prediction for Raising Arizona (1987))
[1]
prediction for Dead Man Walking (1995)
[1]
prediction for Chasing Amy (1997)
[0]


Cross validation: 
 [0.60359116 0.58494475 0.59633978 0.60497238 0.59481865 0.60587219
 0.60207254 0.59136442 0.59378238 0.58411054]
Testing score: 0.5933139936455312


Accuracy modello: 0.5933
Confusion matrix

 [[1900 1455]
 [1489 2395]]
-------------------------------








/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


Average expected loss: 0.498
Average bias: 3602.071
Average variance: 0.174
Sklearn 0-1 loss: 0.407
Average expected loss--After pruning: 0.504
Average bias--After pruning: 3674.198
Average variance--After pruning: 0.070
Sklearn 0-1 loss--After pruning: 0.438


In [ ]:
#@title Settimo test: Naive-Bayes
#@markdown SI Data cleaning iniziale, drop tabelle
#@markdown
#@markdown SI Feature selection, prese le 24 variabili indipendenti più correlate con la variabile target
#@markdown
#@markdown SI Feature Scaling
#@markdown
#@markdown SI Data Balancing, Undersampling
#@markdown
#@markdown SI Undersampling
#@markdown
#@markdown NO Oversampling
X = final_dataset.iloc[:,3:27]
print(X.columns)
y = final_dataset.iloc[:,27:]
print(y.columns)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=0
)

model = GaussianNB()

modello = model.fit(X_train, y_train)

for i in range(1, len(final_dataset)):
    movie = final_dataset.iloc[i, 2]
    if(movie == 'Raising Arizona (1987)'):
        movie_1_ind = final_dataset.iloc[i, 3:27]
        movie_1_real = final_dataset.iloc[i, 27]
    if(movie == 'Little Big League (1994)'):
        movie_2_ind = final_dataset.iloc[i, 3:27]
        movie_2_real = final_dataset.iloc[i, 27]
    if (movie == 'Mighty Aphrodite (1995)'):
        movie_3_ind = final_dataset.iloc[i, 3:27]
        movie_3_real = final_dataset.iloc[i, 27]
print("prediction for Raising Arizona (1987))")
predict = model.predict([movie_1_ind])
print(predict)
print("Real value")
print(movie_1_real)
print("prediction for Dead Man Walking (1995)")
predict = model.predict([movie_2_ind])
print(predict)
print("Real value")
print(movie_2_real)
print("prediction for Chasing Amy (1997)")
predict = model.predict([movie_3_ind])
print(predict)
print("Real value")
print(movie_3_real)
print("\n")

score = cross_val_score(modello, X_train, y_train, cv = 10)
print("Cross validation: \n",score)
print("Testing score: {}".format(modello.score(X_test, y_test)))
print("\n")

y_pred = model.predict(X_test)
accuray = accuracy_score(y_pred, y_test)
print("Accuracy:", accuray)

cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix\n\n', cm)





